In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

# restore saved variables
%store -r summary_misdemeanor2_fl_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model
no stored variable summary_misdemeanor2_fl_inter_model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['misdemeanor_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['misdemeanor_two_year'].values

In [4]:
#### CART
depth = [1,2,3]
impurity = [0.001, 0.003]
split = [2,3,4]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, split, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [5]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.5745913809467573, 0.6470497224122114)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_two_year'].values

## columns 
cols = KY_X.columns

In [7]:
c_grid={'C': [0.03, 0.04, 0.05, 0.06, 0.07]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary

{'best_auc': 0.6083211741919411,
 'best_params': {'C': 0.07},
 'auc_diffs': 0.042578144752915326,
 'KY_score': 0.6199385804272899}

### RiskSLIM

In [22]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_two_year'].values

## columns 
cols = KY_X.columns

In [23]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.05, random_state=816).fit(FL_X, FL_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

14

In [24]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [25]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'misdemeanor_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/20/19 @ 12:08 PM | 193 rows in lookup table
10/20/19 @ 12:08 PM | ------------------------------------------------------------
10/20/19 @ 12:08 PM | runnning initialization procedure
10/20/19 @ 12:08 PM | ------------------------------------------------------------
10/20/19 @ 12:08 PM | CPA produced 2 cuts
10/20/19 @ 12:08 PM | running naive rounding on 4 solutions
10/20/19 @ 12:08 PM | best objective value: 0.6931
10/20/19 @ 12:08 PM | rounding produced 4 integer solutions
10/20/19 @ 12:08 PM | best objective value is 0.6931
10/20/19 @ 12:08 PM | running sequential rounding on 4 solutions
10/20/19 @ 12:08 PM | best objective value: 0.6931
10/20/19 @ 12:08 PM | sequential rounding produced 0 integer solutions
10/20/19 @ 12:08 PM | polishing 4 solutions
10/20/19 @ 12:08 PM | best objective value: 0.6931
10/20/19 @ 12:08 PM | polishing produced 4 integer solutions
10/20/19 @ 12:08 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 12:08 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5820.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

10/20/19 @ 12:08 PM | polishing produced 4 integer solutions
10/20/19 @ 12:08 PM | initialization produced 7 feasible solutions
10/20/19 @ 12:08 PM | best objective value: 0.5801
10/20/19 @ 12:08 PM | ------------------------------------------------------------
10/20/19 @ 12:08 PM | completed initialization procedure
10/20/19 @ 12:08 PM | ------------------------------------------------------------
10/20/19 @ 12:08 PM | 193 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.5490242961962114
CPXPARAM_MIP_Tolerances_UpperCutoff              0.58005631323798246


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 12:08 PM | adding 245 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5801.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 107896  4674        0.5723     9        0.5740        0.5713   534611    0.48%          rho_13 D 107896  61199     27
 110030  3165        cutoff              0.5740        0.5721   540915    0.33%          rho_13 U 110030  64049     27
Elapsed time = 37.67 sec. (41262.76 ticks, tree = 1.44 MB, solutions = 9)
 112085  1479        cutoff              0.5740        0.5731   546603    0.16%          rho_14 D 112085  39358     26

User cuts applied:  758

Root node processing (before b&c):
  Real time             =    0.08 sec. (1.53 ticks)
Sequential b&c:
  Real time             =   38.75 sec. (42917.30 ticks)
                          ------------
Total (root+branch&cut) =   38.83 sec. (42918.83 ticks)
+----------------------------------------------+------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(-2 + score))   |                  |           |
| ============================================ | ================ | ========= |
| p_fta_two_year1                              | 

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 12:09 PM | adding 252 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6005.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

10/20/19 @ 12:10 PM | ------------------------------------------------------------
10/20/19 @ 12:10 PM | runnning initialization procedure
10/20/19 @ 12:10 PM | ------------------------------------------------------------
10/20/19 @ 12:10 PM | CPA produced 2 cuts
10/20/19 @ 12:10 PM | running naive rounding on 4 solutions
10/20/19 @ 12:10 PM | best objective value: 0.6931
10/20/19 @ 12:10 PM | rounding produced 4 integer solutions
10/20/19 @ 12:10 PM | best objective value is 0.6931
10/20/19 @ 12:10 PM | running sequential rounding on 4 solutions
10/20/19 @ 12:10 PM | best objective value: 0.6931
10/20/19 @ 12:10 PM | sequential rounding produced 0 integer solutions
10/20/19 @ 12:10 PM | polishing 4 solutions
10/20/19 @ 12:10 PM | best objective value: 0.6931
10/20/19 @ 12:10 PM | polishing produced 4 integer solutions
10/20/19 @ 12:10 PM | initialization produced 7 feasible solutions
10/20/19 @ 12:10 PM | best objective value: 0.5831
10/20/19 @ 12:10 PM | -----------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 12:10 PM | adding 237 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5831.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

10/20/19 @ 12:10 PM | rounding produced 5 integer solutions
10/20/19 @ 12:10 PM | best objective value is 0.6931
10/20/19 @ 12:10 PM | running sequential rounding on 5 solutions
10/20/19 @ 12:10 PM | best objective value: 0.6931
10/20/19 @ 12:10 PM | sequential rounding produced 0 integer solutions
10/20/19 @ 12:10 PM | polishing 5 solutions
10/20/19 @ 12:10 PM | best objective value: 0.6931
10/20/19 @ 12:10 PM | polishing produced 5 integer solutions
10/20/19 @ 12:10 PM | initialization produced 9 feasible solutions
10/20/19 @ 12:10 PM | best objective value: 0.5855
10/20/19 @ 12:10 PM | ------------------------------------------------------------
10/20/19 @ 12:10 PM | completed initialization procedure
10/20/19 @ 12:10 PM | ------------------------------------------------------------
10/20/19 @ 12:10 PM | 193 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPX

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 12:10 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5855.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [28]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_train_score'])

(0.6221038022924323, 0.5622264950033649)

In [29]:
#### save results
summary_misdemeanor2_fl_inter_model = {"CART": cart_summary,
                                       "EBM": ebm_summary,
                                       "Stumps": stumps_summary,
                                       "RiskSLIM": riskslim_summary}
%store summary_misdemeanor2_fl_inter_model

Stored 'summary_misdemeanor2_fl_inter_model' (dict)


In [30]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Two Year\\"
results = [["Misdemeanor",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Two Year FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)